In [ ]:
# --- Getting Shap-e and Slicing Algorithm Files ---
!git clone https://github.com/openai/shap-e
%cd /content/shap-e/
!pip install -e .
!git clone https://github.com/Nehri/slicing_algorithm.git


In [ ]:
# --- Python Packages Needed ---
!pip install pymeshlab
!pip install numpy
!pip install torch
!pip install numpy-stl
!pip install stlconverter
!pip install colorama

In [3]:
import torch
import pymeshlab

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # This line sets torch to use a GPU for much faster processing

In [ ]:
# --- Set the Stable Diffusion Models ---
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [19]:
# --- Setup Shap-e/Stable Diffusion Setting and User Prompt ---
batch_size = 1
guidance_scale = 15.0
prompt = "baby grand piano" # area for user to type text prompt for mesh generation

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True, # this sets the amount of noise in processing the mesh
    karras_steps=20,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
# --- Write the Generated Mesh to OBJ File ---
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

In [ ]:
# --- Code to convert the OBJ file to STL file ---
import pymeshlab

ms = pymeshlab.MeshSet()

# Load the OBJ file
ms.load_new_mesh('/content/shap-e/mesh_0.obj')

# Save as STL
ms.save_current_mesh('mesh.stl')

In [ ]:
# --- Convert Binary STL File to ASCII STL File ---
!python3 -m stlconverter /content/shap-e/mesh.stl stla

In [ ]:
# --- Slicing STL Mesh with User Provided Parameters: 0.25 is layer height and 0 is infill percentage ---
!python3 /content/slicing_algorithm/slicing.py /content/shap-e/mesh-converted-ASCII.stl 0.25 0